# Datathon 2020 Code
Code used to produce the model

In [ ]:
# imports

import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np
import requests
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
# the given dataset
part1 = pd.read_csv("Part 1.csv").drop(0)
part2 = pd.read_csv("Part 2.csv").drop(0)
part3 = pd.read_csv("Part 3.csv").drop(0)
part4a = pd.read_csv("Part 4a.csv").drop(0)
part4b = pd.read_csv("Part 4b.csv").drop(0)
part5 = pd.read_csv("Part 5.csv").drop(0)
df = pd.concat([part1, part2, part3, part4a, part4b, part5]).reset_index(drop=True)

For each sales range, assume median value and map it

In [ ]:
RCPSZFE_list = ['Establishments operated entire year with sales/receipts/revenue of $100,000 to $249,999',
       'Establishments operated entire year with sales/receipts/revenue of $250,000 to $499,999',
       'Establishments operated entire year with sales/receipts/revenue of $500,000 to $999,999',
       'Establishments operated entire year with sales/receipts/revenue of $1,000,000 or more',
       'Establishments operated entire year with sales/receipts/revenue less than $100,000']
df = df[df['RCPSZFE.display-label'].isin(RCPSZFE_list)]
RCPSZFE_values = [175000, 375000, 750000, 1000000, 50000]
master_dict = {}
for key, value in zip(RCPSZFE_list, RCPSZFE_values):
    master_dict[key] = value
df['RCPSZFE.display-label'] = df['RCPSZFE.display-label'].map(master_dict)
df["ESTAB"] = df["ESTAB"].astype(int)

For each zip code, calculate total revenue generated by all establishments per sector by multiplying the revenue generated with the number of establishments. Save to pickle file because this process will take a long time

In [ ]:
master_list = []
for geoid in tqdm(df["GEO.id"].unique()):
    sample = df[df["GEO.id"] == geoid]
    sample["total"] = sample["RCPSZFE.display-label"]*sample["ESTAB"]
    example = sample.groupby("NAICS.display-label")["total"].sum()
    example = pd.DataFrame({geoid: example}).transpose()
    master_list.append(example)
pickle.dump(master_list, open("Datathon.p", "wb"))

Concat all to a single dataframe to get a dataframe that describes per zip code, save to pickle again as this will also take a long time

In [ ]:
master_df = pd.concat(master_list)
pickle.dump(master_df, open("Datathon_master_d.p", "wb"))

In [ ]:
master_df = pickle.load(open("Datathon_master_df.p", "rb"))
master_df = master_df.fillna(0)
master_df.index = master_df.index.str[-5:].astype(int)

External data source: https://www.psc.isr.umich.edu/dis/census/Features/tract2zip/
Mean and median HH income per zip and population per zip. Initial preprocessing to calculate total capitol by multiplying avg income per person with population

In [ ]:
zip_info = pd.read_csv("MedianZIP.csv")
zip_info["Median"] = zip_info["Median"].str.replace(",", "")
zip_info["Pop"] = zip_info["Pop"].str.replace(",", "")
zip_info["total capitol"] = zip_info["Median"].astype(int)*zip_info["Pop"].astype(int)/2.52
zip_info.loc[zip_info["Zip"].astype(str).apply(lambda x: len(x) != 5), "Zip"] = "0" + zip_info[zip_info["Zip"].astype(str).apply(lambda x: len(x) != 5)]["Zip"].astype(str)
zip_info.index = zip_info["Zip"].astype(int).values
zip_info.drop("Zip", axis=1, inplace=True)

Combined data frame, calculate potential for revenue by subtracting how much money went to businesses from how much income was generated overall per zip code

In [ ]:
new_df = master_df.join(zip_info).dropna()
new_df["total spending"] = master_df.sum(axis=1)
new_df["potential"] = new_df["total capitol"] - new_df["total spending"]

Customizable list of sectors of interest and weights

In [ ]:
PG = [["Children's and infants' clothing stores",
      'Convenience stores',
      'Cosmetics, beauty supplies, and perfume stores',
      'General merchandise stores',
      'Grocery stores',
      'Health and personal care stores' ,
      'Household appliance stores',
      'Pharmacies and drug stores' ,
      'Supermarkets and other grocery (except convenience) stores',
      'Warehouse clubs and supercenters', 1]]

JD = [['Fruit and vegetable markets',
      'Lawn and garden equipment and supplies stores',
      'Nursery, garden center, and farm supply stores', 0.65],
      ['Automotive parts and accessories stores',
      'Automotive parts, accessories, and tire stores',
      'Motor vehicle and parts dealers',
      'Motorcycle, boat, and other motor vehicle dealers', 0.1],
      ['Building material and garden equipment and supplies dealers',
      'Building material and supplies dealers',
      'Other building material dealers',
      'Outdoor power equipment stores', 0.25]]

SF = [['Automobile dealers',
      'Boat dealers',
      'Recreational vehicle dealers',
      'Used car dealers',
      'New car dealers', 0.7],
      ["Children's and infants' clothing stores",
      'Pharmacies and drug stores', 0.2],
      ['Home furnishings stores',
      'Jewelry stores',
      'Pet and pet supplies stores', 0.1]]

SN = [['Electronic shopping',
      'Electronic shopping and mail-order houses', 1]]

SNL = [["total spending", 1]]

Main function to get the zip codes: sector_info is the list of sectors and their weights, n is the number of zip codes to generate. Using the categories and the weights, we calculate marketing attractiveness, which is how much of the economy the company should be interested in. Then we calculate the percentage for market attractiveness by dividing by the total spending economy for the zip code. Then, by multiplying by total spending, we get the total percentage of the economic potential that the company is looking to gain by advertising to that specific zip code. By sorting by this metric, we can get a top n prioritized list of zip codes to advertise to.

In [ ]:
def get_zipcodes(sector_info, n=100):
    total = 0
    for val in sector_info:
        total += new_df[val[:-1]].sum(axis=1)*val[-1]
    new_df["marketing attractiveness"] = total
    new_df["marketing attractiveness percentage"] = new_df["marketing attractiveness"]/new_df["total spending"]
    new_df["metric"] = new_df["marketing attractiveness percentage"]*new_df["potential"]
    zip_codes = new_df["metric"].sort_values(ascending=False).head(n).index
    return zip_codes, new_df[["marketing attractiveness percentage", "potential", "metric"]]

Run the function. Example: P&G

In [ ]:
zip_codes, temp = get_zipcodes(PG, 20)

Able to view the resulting data table

In [ ]:
temp.sort_values(by="metric", ascending=False).head(20)